# Bhejane Online Trading Store


<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem. 
 
**Your mission, should you choose to accept it:**

There is a denormalised database consisting of two tables. The task here is to transform the database such that it's in third normal form (3NF). To ensure that a consistent normalisation process is followed, there is an Entity Relationship Diagram (ERD) which is a guideline on what tables need to be produced.

Though the normalisation process, you will be guided into acknowledging client (Bhenjane Trading) use-cases for the database, and dealing with data anomalies in SQL. 

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
Please use the below command to install sql_magic, this is the package that will assist with SQL syntax hightlighting.
* pip install sql_magic

Remember to start each new cell with " %%read_sql "

In [2]:
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [3]:
# Load data
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [3]:
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [138]:
%%read_sql
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 09:50:52 AM GMT Daylight Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

In [139]:
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Explore the Denormalized Tables
SQL queries to explore properties of the dataset.i.e Look for data inconsistencies, anormalies, redundancies etc to guide your normalization process

In [140]:
%%read_sql

SELECT 
    * 
FROM 
    Products 
LIMIT 10;

Query started at 09:50:56 AM GMT Daylight Time; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,1 x Hikvision 1080P Bullet camera,399.0
1,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,Manual,399.0
2,,,,10325354918,467.0,ZEE,Fashion / Accessories / Scarves,Grey,,ZEE 3-in-1 Unisex Gaiter,,0.0,Limited (6 months),,,139.0
3,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Makeup / Face / Found...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
4,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Shop By Brand / Estee...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
5,,,,27131187035,275.0,Estee Lauder,Beauty / Makeup / Face / Foundation,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
6,,,,39800085139,247.0,Energizer,Cameras / Cameras & Lenses / Camera Accessorie...,,South Africa,Energizer 3V CR2450 Lithium Coin Battery,,0.0,Limited (12 months),,,49.0
7,,,,43859499182,386.0,Fellowes,Office & Stationery / Stationery / Office Supp...,,South Africa,Fellowes HFC Free Air Duster - 350ml,,0.0,Limited (6 months),,1 x 350ml Free Air Duster,149.0
8,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0
9,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the target ERD create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


It is important to create all tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form

In [141]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 09:51:00 AM GMT Daylight Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [142]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
    SELECT DISTINCT
         Barcode
        ,NavigationPath
        ,ItemDescription
        ,Colour
        ,ProductDescription
        ,Brand
        ,Price
        ,Quantity
        ,PackType
        ,Warranty
        ,StockCountry
        ,Weight_kg
        ,Volume_litre
        ,Length
        ,Width
        ,Height 
    FROM 
        Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
    SELECT DISTINCT 
        CartID
        ,Barcode
        ,UserName
        ,InvoiceDate
        ,Total
    FROM
        Transactions;


Query started at 09:51:03 AM GMT Daylight Time; Query executed in 0.00 m

In [143]:
%%read_sql

SELECT 
    * 
FROM 
    transactions_1NF 
LIMIT 4;

Query started at 09:51:04 AM GMT Daylight Time; Query executed in 0.00 m

,CartID,Barcode,UserName,InvoiceDate,Total
0,102,300507946,DIMPHO,2020-07-02 0:00:00,1523.0
1,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
2,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
3,179,617566827837,Zanele,2020-04-04 0:00:00,3464.0


###  7. Convert the database into its 2nd Normal Form (2NF).

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will separate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`.

#### 7.3 Data Anomalies

You may stumble into an error when you insert the data which violates this constraint. You query this with the data owner, and you are told that there are three transactions in the database which were errors. You are informed that *all transactions in the database correspond to products in the database*.

These entries were captured incorrectly, and must be removed. How will you deal with this? 

In [146]:
%%read_sql

SELECT 
    * 
FROM 
    Transactions_1NF 
WHERE 
    barcode NOT IN 
        (SELECT barcode FROM products_1NF)

Query started at 09:51:10 AM GMT Daylight Time; Query executed in 0.00 m

,CartID,Barcode,UserName,InvoiceDate,Total
0,10,889899982693,Tanya,2020-05-25 0:00:00,220.0
1,62,6007226069631,Angelica Fernandez,2020-05-29 0:00:00,99.0
2,105,6007226069631,Zelda,2020-04-26 0:00:00,687.0


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [147]:
%%read_sql
DELETE 
FROM 
    Transactions_1NF 
WHERE 
    barcode = '889899982693' OR barcode = '6007226069631'

Query started at 09:51:13 AM GMT Daylight Time; Query executed in 0.00 m

### Check if the delete query worked

In [148]:
%%read_sql
-- Should show no entries!
SELECT 
    COUNT(DISTINCT barcode) 
FROM 
    Transactions_1NF 
WHERE 
    barcode NOT IN
        (SELECT barcode FROM products_1NF)

Query started at 09:51:15 AM GMT Daylight Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that one understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint oneself!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 

One will need to work around this by removing the erroneous data from the tables before continuing, since this would violate the FK constraint between the Transactions_2NF and Products_2NF table otherwise. 

_Hint: Identify the observations in the Transactions-1NF table, and remove them from there_

#### 7.5 Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database. 

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. You are tasked with removing 'useless' entries as you take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

_Hint: Make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [149]:
%%read_sql 

SELECT 
    DISTINCT ItemDescription,
    PackType,
    Warranty 
FROM
    Products
WHERE 
    ItemDescription = '';

Query started at 09:51:22 AM GMT Daylight Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


### Let's do it! Construct the database above in second normal form

In [150]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 09:51:28 AM GMT Daylight Time; Query executed in 0.00 m

### Populate the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And insert only the relevant entries. 

In [151]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
    SELECT DISTINCT 
        NavigationPath 
    FROM
        Products_1NF;

INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
    SELECT DISTINCT 
        ItemDescription,
        PackType,
        Warranty 
    FROM
        Products_1NF
    WHERE 
        ItemDescription <> ''
            ;

INSERT INTO "Colours_2NF"("Colour")
    SELECT DISTINCT 
        Colour
    FROM
        Products_1NF
    WHERE 
        Colour <> ''
        ;

Query started at 09:51:31 AM GMT Daylight Time; Query executed in 0.00 m

A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [152]:
%%read_sql

SELECT 
    * 
FROM 
    Products_2NF 
LIMIT 2;

Query started at 09:51:35 AM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Brand,Price,Quantity,StockCountry,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID


In [153]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
    SELECT
        PathID,
        ItemID,
        ColourID,
        Products_1NF.Barcode,
        ProductDescription,
        Brand,
        Price,
        Quantity,
        StockCountry,
        Weight_kg, 
        Volume_litre, 
        Length,
        Width,
        Height 
    FROM 
        Products_1NF
    LEFT JOIN 
        Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
    LEFT JOIN 
        PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
    LEFT JOIN 
        Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 09:51:45 AM GMT Daylight Time; Query executed in 0.00 m

In [154]:
%%read_sql

SELECT 
    * 
FROM 
    Navigation_2NF 
LIMIT 5;

Query started at 09:51:48 AM GMT Daylight Time; Query executed in 0.00 m

,PathID,NavigationPath
0,1,Fashion / Accessories / Scarves
1,2,Health / Personal Care / Lip & Skin Care / Bod...
2,3,Cellphones & Wearables / Cellular Accessories ...
3,4,Computers & Tablets / Laptops / Notebooks
4,5,Fashion / Accessories


Last but not least,insert into the `Transactions_2NF`table.

In [155]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
    SELECT DISTINCT 
        CartID||Transactions_1NF.Barcode||UserName
        ,CartID
        ,Transactions_1NF.Barcode
        ,UserName
        ,InvoiceDate
        ,Total
    FROM
        Transactions_1NF
    LEFT JOIN 
        Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 09:51:53 AM GMT Daylight Time; Query executed in 0.00 m

In [156]:
%%read_sql

SELECT 
    * 
FROM 
    Transactions_2NF 
LIMIT 5;

Query started at 09:51:56 AM GMT Daylight Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0


In [157]:
%%read_sql

SELECT 
    * 
FROM 
    Products_2NF 
LIMIT 5;

Query started at 09:51:58 AM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Brand,Price,Quantity,StockCountry,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,,0.0,,,,11,1.0,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,,0.0,,,,11,2.0,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,ZEE,139.0,467,,,0.0,,,,1,NaN,1.0
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,,0.0,,,,8,NaN,2.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,,0.0,,,,9,NaN,2.0


<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

Label your tables as they appear in the ERD sketch. In the same fashion as done in 2NF, note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [166]:
%%read_sql

--#Create tables required for 3NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigations_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 09:54:38 AM GMT Daylight Time; Query executed in 0.00 m

In [167]:
%%read_sql
CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150),
    "Barcode" VARCHAR(150) NOT NULL,
    "UserID" INTEGER,
    "CartID" INTEGER,
    PRIMARY KEY("CartID||Barcode||UserName"),
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_3NF ('Barcode'),
    CONSTRAINT fk_trans FOREIGN KEY('UserID') REFERENCES Users_3NF ('UserID'),
    CONSTRAINT fk_trans FOREIGN KEY('CartID') REFERENCES Carts_3NF ('CartID')  
);
    
CREATE TABLE "Navigations_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);

CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR(150),
    PRIMARY KEY("BrandID" AUTOINCREMENT)
);

CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    PRIMARY KEY("LocationID" AUTOINCREMENT)
);

Query started at 09:54:42 AM GMT Daylight Time; Query executed in 0.00 m

In [168]:
%%read_sql

--# Populate the tables so to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigations_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
    SELECT DISTINCT UserName FROM Transactions_2NF;


Query started at 09:54:45 AM GMT Daylight Time; Query executed in 0.00 m

In [4]:
%%read_sql

SELECT 
    * 
FROM 
    Users_3NF 
LIMIT 5;

Query started at 03:56:26 PM GMT Daylight Time; Query executed in 0.00 m

,UserID,UserName
0,1,Hendrik
1,2,Faristha
2,3,JOHAN
3,4,Sandra
4,5,Sheila


### <font color='turquoise'>Action: proceed with the insertion into `Carts_3NF`.   </font>

Take note that there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [169]:
%%read_sql
SELECT 
    * 
FROM 
    Transactions_2NF
LIMIT 5;

Query started at 09:54:48 AM GMT Daylight Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0


In [170]:
%%read_sql

INSERT INTO "Carts_3NF"("CartID","InvoiceDate","Total")
    SELECT DISTINCT CartID, InvoiceDate, Total FROM Transactions_2NF;

Query started at 09:54:51 AM GMT Daylight Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into the tables below. </font>

Similarly to the table above, mind the occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows. 


In [171]:
%%read_sql

INSERT INTO "Locations_3NF" ("StockCountry")
    SELECT DISTINCT StockCountry FROM Products_2NF WHERE StockCountry <> '';

INSERT INTO "Brands_3NF" ("Brand")
    SELECT DISTINCT Brand FROM Products_2NF WHERE Brand <> '';
    
INSERT INTO "Colours_3NF" ("Colour")
    SELECT DISTINCT Colour FROM Colours_2NF;

INSERT INTO "PackageContents_3NF" ("ItemDescription","PackType","Warranty")
    SELECT DISTINCT ItemDescription, PackType, Warranty FROM PackageContents_2NF;

INSERT INTO "Navigations_3NF" ("NavigationPath")
    SELECT DISTINCT NavigationPath FROM Navigation_2NF;



Query started at 09:54:55 AM GMT Daylight Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Products_3NF`. </font>
Take note of the joins required for additional data contained in other tables. 

In [175]:
%%read_sql

INSERT INTO "Products_3NF"("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
    SELECT
        Navigations_3NF.PathID ,
        PackageContents_3NF.ItemID,
        Colours_3NF.ColourID,
        BrandID,
        LocationID,
        Barcode,
        ProductDescription,
        Price,
        Quantity,
        Weight_kg,
        Volume_litre,
        Length,
        Width,
        Height
    FROM
        Products_2NF
    LEFT JOIN 
        Navigations_3NF ON Products_2NF.PathID = Navigations_3NF.PathID 
    LEFT JOIN 
        PackageContents_3NF ON Products_2NF.ItemID = PackageContents_3NF.ItemID
    LEFT JOIN 
        Colours_3NF ON Products_2NF.ColourID = Colours_3NF.ColourID
    LEFT JOIN 
        Locations_3NF ON Products_2NF.StockCountry = Locations_3NF.StockCountry
    LEFT JOIN 
        Brands_3NF ON Products_2NF.Brand = Brands_3NF.BrandID;


Query started at 09:55:51 AM GMT Daylight Time; Query executed in 0.00 m

In [176]:
%%read_sql

SELECT 
    * 
FROM 
    Products_3NF 
LIMIT 5;

Query started at 09:56:02 AM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,1.0,NaN,None,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,2.0,NaN,None,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,139.0,467,,0.0,,,,1,NaN,1.0,None,NaN
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,8,NaN,2.0,None,1.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,9,NaN,2.0,None,1.0


In [24]:
%%read_sql

SELECT 
    * 
FROM 
    Products_3NF 
LIMIT 5;

Query started at 12:21:54 PM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,1.0,NaN,1,1
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,2.0,NaN,2,2
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,139.0,467,,0.0,,,,1,NaN,1.0,3,3
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,8,NaN,2.0,4,4
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,9,NaN,2.0,5,5


In [64]:
%%read_sql

SELECT 
    * 
FROM 
    Brands_3NF 
LIMIT 20;

Query started at 11:52:07 PM GMT Daylight Time; Query executed in 0.00 m

,BrandID,Brand
0,1,Hikvision
1,2,ZEE
2,3,Estee Lauder
3,4,Energizer
4,5,Fellowes
5,6,WAHL
6,7,Bombay Sapphire
7,8,
8,9,Digitronics
9,10,Intelli-Vision Technology


In [25]:
%%read_sql

SELECT 
    * 
FROM 
    Transactions_2NF 
LIMIT 5;

Query started at 12:24:12 PM GMT Daylight Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0


### <font color='turquoise'>Action: proceed with the insertion into `Transactions_3NF`.   </font>

Follow previous proceedure, and checks. 

In [187]:
%%read_sql

INSERT INTO "Transactions_3NF"("CartID||Barcode||UserName","CartID","Barcode","UserID")
    SELECT DISTINCT 
        Carts_3NF.CartID||Products_3NF.Barcode||Users_3NF.UserName
        ,Carts_3NF.CartID
        ,Products_3NF.Barcode
        ,Users_3NF.UserId
    FROM
        Transactions_2NF 
    LEFT JOIN 
        Carts_3NF ON Transactions_2NF.CartID = Carts_3NF.CartID
    LEFT JOIN 
        Products_3NF ON Transactions_2NF.Barcode = Products_3NF.Barcode
    LEFT JOIN 
        Users_3NF ON Transactions_2NF.UserName = Users_3NF.UserName
    ;

   
   -- Worked but dangerous
   -- Carts_3NF.CartID||Products_3NF.Barcode||UserName
      --,Carts_3NF.CartID
        --,Products_3NF.Barcode
       --,UserId
    --FROM
        --Transactions_2NF 
    --LEFT JOIN Transactions_3NF ON  Transactions_2NF.CartID||Transactions_2NF.Barcode||UserName = Transactions_3NF.CartID||Transactions_3NF.Barcode||UserName
   -- ;

Query started at 10:53:47 AM GMT Daylight Time; Query executed in 0.00 m

In [123]:
%%read_sql

SELECT 
    * 
FROM 
    Transactions_2NF 
LIMIT 6;

Query started at 02:18:10 PM GMT Daylight Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0
5,6718037855448Zanele,6,718037855448,Zanele,2020-06-01 0:00:00,840.0


### <font color='turquoise'>Action: Investigate the DB.   </font>

Note - the tables counts below are what you should arrive at. If you don't, reconsider how you have done the inserts above. Having the tables correctly formatted will allow to answer the questions more easily, and without errors.

| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [189]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
SELECT 
    'Products_3NF' AS table_name, 
    COUNT(*) 
FROM 
    Products_3NF;


Query started at 10:54:02 AM GMT Daylight Time; Query executed in 0.00 m

,table_name,count(*)
0,Products_3NF,1214


In [188]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
SELECT 
    'Transactions_3NF' AS table_name, 
    COUNT(*) 
FROM 
    Transactions_3NF

Query started at 10:53:56 AM GMT Daylight Time; Query executed in 0.00 m

,table_name,count(*)
0,Transactions_3NF,275


In [190]:
%%read_sql 

-- How many entries are here in each of the tables in the databASe now? 
SELECT 'Products_3NF' AS table_name, COUNT(*) FROM Products_3NF
UNION
SELECT 'Transactions_3NF' AS table_name, COUNT(*) FROM Transactions_3NF
UNION
SELECT 'Users_3NF' AS table_name, COUNT(*) FROM Users_3NF
UNION
SELECT 'Navigation_3NF' AS table_name, COUNT(*) FROM Navigation_3NF
UNION
SELECT 'PackageContents_3NF' AS table_name, COUNT(*) FROM PackageContents_3NF
UNION
SELECT 'Colours_3NF' AS table_name, COUNT(*) FROM Colours_3NF
UNION
SELECT 'Brands_3NF' AS table_name, COUNT(*) FROM Brands_3NF
UNION
SELECT 'Locations_3NF' AS table_name, COUNT(*) FROM Locations_3NF
UNION
SELECT 'Carts_3NF' AS table_name, COUNT(*) FROM Carts_3NF


Query started at 10:54:07 AM GMT Daylight Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,396
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


In [35]:
%%read_sql

SELECT 
    * 
FROM 
    Products_3NF 
LIMIT 5;

Query started at 04:44:17 AM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,1.0,NaN,None,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,2.0,NaN,None,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,139.0,467,,0.0,,,,1,NaN,1.0,None,NaN
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,8,NaN,2.0,None,1.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,9,NaN,2.0,None,1.0


In [40]:
%%read_sql

SELECT 
    * 
FROM 
    PackageContents_3NF 
LIMIT 100;

Query started at 04:46:49 AM GMT Daylight Time; Query executed in 0.00 m

,ItemID,ItemDescription,PackType,Warranty
0,1,1 x Hikvision 1080P Bullet camera,,Limited (6 months)
1,2,Manual,,Limited (6 months)
2,3,1 x 350ml Free Air Duster,,Limited (6 months)
3,4,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,Limited (6 months)
4,5,Wahl Easy Cut 15 Piece Complete Hair Clipper Kit,,Limited (6 months)
...,...,...,...,...
95,96,2 x 2032,,Limited (24 months)
96,97,J&B - Rare Scotch Whisky - 750ml,Single,Non-Returnable
97,98,x 1 750ml Glenfiddich - 12 Year Old Special Re...,Single,Non-Returnable
98,99,"x 1 Mixing Bowl 3,0lt",,Limited (6 months)


In [38]:
%%read_sql

SELECT 
    * 
FROM 
    Users_3NF 
LIMIT 5;

Query started at 04:45:46 AM GMT Daylight Time; Query executed in 0.00 m

,UserID,UserName
0,1,Hendrik
1,2,Faristha
2,3,JOHAN
3,4,Sandra
4,5,Sheila


<a id='MCQ_questions'></a>
# 9. MCQ Questions

Having completed the normalisation of the database, you may use the following cells to help you answer these MCQs.

### Q1) How many unique products does the company have?

In [135]:
%%read_sql

SELECT 
    count(DISTINCT(Barcode)) AS Num_Of_Unique_Products 
FROM 
    Products_3NF;

Query started at 07:40:47 AM GMT Daylight Time; Query executed in 0.00 m

,Num_Of_Unique_Products
0,608


### Q2) How many users bought from Bhejane in April 2020?

In [122]:
%%read_sql
-- Connect Users_3NF with Transactions tables on UserID, AND Transactions_3NF with Carts_3NF on CartID
CREATE VIEW trans_users_cartss AS
SELECT 
    Transactions_3NF.Barcode, 
    InvoiceDate, Username, 
    Users_3NF.UserID, 
    Price, 
    Total 
FROM 
    Transactions_3NF
JOIN 
    Users_3NF ON Transactions_3NF.UserID = Users_3NF.UserID
JOIN 
    Carts_3NF ON Transactions_3NF.CartID = Carts_3NF.CartID
JOIN 
    Products_3NF ON Transactions_3NF.Barcode = Products_3NF.Barcode
WHERE 
    Carts_3NF.InvoiceDate LIKE '2020-04-%';


Query started at 07:25:52 AM GMT Daylight Time; Query executed in 0.00 m

In [149]:
%%read_sql

SELECT 
    COUNT(*) 
FROM 
    Carts_3NF 
WHERE 
    InvoiceDate LIKE '2020-04-%';

Query started at 08:32:11 AM GMT Daylight Time; Query executed in 0.00 m

,count(*)
0,35


In [126]:
%%read_sql

SELECT
    COUNT(DISTINCT(UserID)) AS April_Customers 
FROM 
    trans_users_cartss;

Query started at 07:31:19 AM GMT Daylight Time; Query executed in 0.00 m

,April_Customers
0,31


In [150]:
%%read_sql

SELECT 
    COUNT(DISTINCT(Transactions_3NF.UserID)) 
FROM 
    Carts_3NF 
JOIN 
    Transactions_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
WHERE 
    InvoiceDate BETWEEN '2020-04-01' AND '2020-04-31';

Query started at 08:32:29 AM GMT Daylight Time; Query executed in 0.00 m

,count(DISTINCT(Transactions_3NF.UserID))
0,31


### Q3) How many users bought 3 or more items that cost more than R1000?

In [121]:
%%read_sql
    
SELECT 
    DISTINCT(Products_3NF.Barcode), 
    Products_3NF.Price, 
    Carts_3NF.InvoiceDate, 
    Users_3NF.UserName, 
    Carts_3NF.CartID, 
    Carts_3NF.Total 
FROM 
    Products_3NF 
JOIN 
    Transactions_3NF ON Transactions_3NF.Barcode = Products_3NF.Barcode
JOIN 
    Carts_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
JOIN 
    Users_3NF ON  Users_3NF.UserID = Transactions_3NF.UserID 
WHERE 
    Price <> Total AND Total > 1000;

Query started at 12:07:34 AM GMT Daylight Time; Query executed in 0.00 m

,Barcode,Price,InvoiceDate,UserName,CartID,Total
0,4549292118766,899.0,2020-06-09 0:00:00,Hanno,101,1334.0
1,6001878010171,269.0,2020-06-09 0:00:00,Hanno,101,1334.0
2,MPTAL00215519,166.0,2020-06-09 0:00:00,Hanno,101,1334.0
3,300507946,399.0,2020-07-02 0:00:00,DIMPHO,102,1523.0
4,6002322009727,899.0,2020-07-02 0:00:00,DIMPHO,102,1523.0
5,8888021200980,39.0,2020-07-02 0:00:00,DIMPHO,102,1523.0
6,MPTAL00215561,186.0,2020-07-02 0:00:00,DIMPHO,102,1523.0
7,6003000640348,1775.0,2020-08-11 0:00:00,Melandi,113,2535.0
8,8001841126586,449.0,2020-08-11 0:00:00,Melandi,113,2535.0
9,MPTALP14631,311.0,2020-08-11 0:00:00,Melandi,113,2535.0


In [123]:
%%read_sql
    
SELECT 
    COUNT(DISTINCT(Users_3NF.UserName))
FROM 
    Products_3NF 
JOIN 
    Transactions_3NF ON Transactions_3NF.Barcode = Products_3NF.Barcode
JOIN 
    Carts_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
JOIN 
    Users_3NF ON  Users_3NF.UserID = Transactions_3NF.UserID 
WHERE 
    Price <> Total AND Total > 1000;

Query started at 12:11:08 AM GMT Daylight Time; Query executed in 0.00 m

,count(DISTINCT(Users_3NF.UserName))
0,11


### Q4) Which user made the largest purchase on a single transaction?

In [8]:
%%read_sql

SELECT 
    Username, 
    Quantity, 
    Price, 
    Total 
FROM 
    tran_user_cart 
ORDER BY 
    Total DESC,
    Price DESC 
LIMIT 5;

Query started at 09:13:24 PM GMT Daylight Time; Query executed in 0.00 m

,UserName,Quantity,Price,Total
0,Mandla,17,10999.0,10999.0
1,Brigette,17,10999.0,10999.0
2,Andreas Peter,374,3765.0,3765.0
3,Mzoxolo,374,3765.0,3765.0
4,Zanele,19,2599.0,3464.0


### Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [21]:
%%read_sql

SELECT 
    COUNT(*) 
FROM 
    Products_3NF 
WHERE 
    Barcode = 'MPTAL57588104';

Query started at 09:28:34 PM GMT Daylight Time; Query executed in 0.00 m

,count(*)
0,8


### Q6) How many brands are available at Bhejane?

In [39]:
%%read_sql

SELECT 
    COUNT(Brand) 
FROM 
    Brands_3NF;

Query started at 09:52:27 PM GMT Daylight Time; Query executed in 0.00 m

,count(Brand)
0,232


In [23]:
%%read_sql

SELECT 
    COUNT(DISTINCT(Brand)) 
FROM 
    Brands_3NF;

Query started at 09:29:50 PM GMT Daylight Time; Query executed in 0.00 m

,count(DISTINCT(Brand))
0,232


### Q7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [26]:
%%read_sql
SELECT 
    * 
FROM 
    Products_3NF 
WHERE 
    ProductDescription = 'Verimark - Floorwiz 2in1 Mop';

Query started at 09:33:27 PM GMT Daylight Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,537,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,236,None,None,1
1,538,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,237,None,None,1
2,539,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,238,None,None,1


In [138]:
%%read_sql

SELECT 
    DISTINCT(Price)
FROM 
    Products_3NF 
WHERE 
    ProductDescription = 'Verimark - Floorwiz 2in1 Mop';

Query started at 12:26:02 AM GMT Daylight Time; Query executed in 0.00 m

,Price
0,179.0


### Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [28]:
%%read_sql

SELECT 
    ProductDescription, 
    Length, 
    Width, 
    Height, 
    (Length*Width*Height) AS Volume 
FROM 
    Products_3NF
WHERE 
    ProductDescription = 'Russell Hobbs - Slow Cooker';

Query started at 09:37:34 PM GMT Daylight Time; Query executed in 0.00 m

,ProductDescription,Length,Width,Height,Volume
0,Russell Hobbs - Slow Cooker,24.0,34.0,"33,4",26928.0
1,Russell Hobbs - Slow Cooker,24.0,34.0,"33,4",26928.0
2,Russell Hobbs - Slow Cooker,24.0,34.0,"33,4",26928.0
3,Russell Hobbs - Slow Cooker,24.0,34.0,"33,4",26928.0


### Q9) Which user made the most transactions in the Year 2020?

In [33]:
%%read_sql

SELECT 
    Users_3NF.UserName, 
    COUNT(Users_3NF.UserName) 
FROM 
    Transactions_3NF 
JOIN 
    Users_3NF ON Users_3NF.UserID = Transactions_3NF.UserID
GROUP BY 
    Users_3NF.UserName 
ORDER BY 
    COUNT(Users_3NF.UserName) DESC
LIMIT 5;

Query started at 09:46:48 PM GMT Daylight Time; Query executed in 0.00 m

,UserName,count(Users_3NF.UserName)
0,Cornelis,9
1,Erlo,8
2,Zanele,6
3,Heather,6
4,Mandla,5


Q10) What is the total number of users that shop at Bhejane?

In [40]:
%%read_sql

SELECT 
    COUNT(DISTINCT(UserName)) 
FROM 
    Users_3NF;

Query started at 09:55:41 PM GMT Daylight Time; Query executed in 0.00 m

,count(DISTINCT(UserName))
0,158


In [139]:
%%read_sql

SELECT 
    COUNT(UserName) 
FROM 
    Users_3NF;

Query started at 12:29:54 AM GMT Daylight Time; Query executed in 0.00 m

,count(UserName)
0,158


Q11) What is the record count for the Colours_3NF Table?

In [46]:
%%read_sql

SELECT 
    COUNT(*) AS Total_Colour 
FROM 
    Colours_3NF;

Query started at 10:06:21 PM GMT Daylight Time; Query executed in 0.00 m

,Total_Colour
0,17


In [47]:
%%read_sql

SELECT 
    COUNT(DISTINCT(ColourID)) AS Total_Colour 
FROM 
    Products_3NF 
WHERE 
    ColourID IS NOT NULL;

Query started at 10:06:38 PM GMT Daylight Time; Query executed in 0.00 m

,Total_Colour
0,17


Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [48]:
%%read_sql

SELECT 
    Price 
FROM 
    Products_3NF 
WHERE 
    Barcode IN ('MPTAL57588104', '5000394203921', '6932391917652');

Query started at 10:10:56 PM GMT Daylight Time; Query executed in 0.00 m

,Price
0,35.0
1,329.0
2,329.0
3,329.0
4,370.0
5,370.0
6,370.0
7,370.0
8,370.0
9,370.0


In [50]:
print(370 * 8)

2960


In [51]:
print(329 * 3)

987


In [52]:
print(2960 + 987 + 35)

3982


Q13) What is the barcode of the most sold product?

In [146]:
%%read_sql

SELECT 
    COUNT(Products_3NF.Barcode), 
    Products_3NF.Barcode 
FROM 
    Transactions_3NF
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
--WHERE Products_3NF.Barcode = '4015400541813' 
GROUP BY 
    Products_3NF.Barcode 
ORDER BY 
    COUNT(Products_3NF.Barcode) DESC 
LIMIT 60;

Query started at 12:36:42 AM GMT Daylight Time; Query executed in 0.00 m

,count(Products_3NF.Barcode),Barcode
0,16,6003000640348
1,12,MPTAL00451356
2,12,4902505163104
3,10,6005427110459
4,10,4549292118766
5,10,4549292072389
6,9,6002322009727
7,8,MPTAL57588104
8,8,6009706869533
9,8,6002305001472


In [148]:
%%read_sql

SELECT 
    Products_3NF.Barcode, COUNT(Products_3NF.Barcode) 
FROM 
    Transactions_3NF
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
WHERE 
    Products_3NF.Barcode = '4015400541813' ;

Query started at 12:38:52 AM GMT Daylight Time; Query executed in 0.00 m

,Barcode,count(Products_3NF.Barcode)
0,4015400541813,4


In [149]:
%%read_sql

SELECT 
    Products_3NF.Barcode, 
    COUNT(Products_3NF.Barcode) 
FROM 
    Transactions_3NF 
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
WHERE 
    Products_3NF.Barcode = '6001865825405';

Query started at 12:39:13 AM GMT Daylight Time; Query executed in 0.00 m

,Barcode,count(Products_3NF.Barcode)
0,6001865825405,3


In [150]:
%%read_sql

SELECT 
    Products_3NF.Barcode, 
    COUNT(Products_3NF.Barcode) 
FROM 
    Transactions_3NF 
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
WHERE 
    Products_3NF.Barcode = '6001106124069';

Query started at 12:39:23 AM GMT Daylight Time; Query executed in 0.00 m

,Barcode,count(Products_3NF.Barcode)
0,6001106124069,3


In [151]:
%%read_sql

SELECT 
    Products_3NF.Barcode, 
    COUNT(Products_3NF.Barcode) 
FROM 
    Transactions_3NF 
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
WHERE 
    Products_3NF.Barcode = '6009686620537';

Query started at 12:39:30 AM GMT Daylight Time; Query executed in 0.00 m

,Barcode,count(Products_3NF.Barcode)
0,6009686620537,3


Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [79]:
%%read_sql

SELECT 
    Products_3NF.ProductDescription, 
    Transactions_3NF.Barcode, 
    Carts_3NF.CartID,
    Users_3NF.UserID, 
    Users_3NF.UserName
FROM 
    Transactions_3NF 
JOIN 
    Products_3NF ON Products_3NF.Barcode = Transactions_3NF.Barcode
JOIN 
    Carts_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
JOIN 
    Users_3NF ON  Users_3NF.UserID = Transactions_3NF.UserID 
WHERE 
    Users_3NF.UserName = 'Cornelis' AND Carts_3NF.InvoiceDate = '2020-06-28 0:00:00';

Query started at 10:56:04 PM GMT Daylight Time; Query executed in 0.00 m

,ProductDescription,Barcode,CartID,UserID,UserName
0,Dettol Body Wash - Shower Gel - Original - 600ml,6001106124069,104,9,Cornelis
1,Vitality Aloe Vera Gel 100ml,6003753002035AP,104,9,Cornelis
2,Vitality Aloe Vera Gel 100ml,6003753002035AP,104,9,Cornelis
3,Vitality Aloe Vera Gel 100ml,6003753002035AP,104,9,Cornelis
4,Sparq Active Resistance Loop Band - Set of 5 w...,MPTAL00555756,104,9,Cornelis
5,Sparq Active Resistance Loop Band - Set of 5 w...,MPTAL00555756,104,9,Cornelis
6,Protective Face Shield,MPTAL72193414,104,9,Cornelis
7,Protective Face Shield,MPTAL72193414,104,9,Cornelis


Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [92]:
%%read_sql

SELECT 
    DISTINCT(Users_3NF.UserName), 
    Locations_3NF.StockCountry, 
    Products_3NF.ColourID, 
    Colours_3NF.Colour, 
    Carts_3NF.Total, 
    Carts_3NF.InvoiceDate
FROM 
    Products_3NF 
JOIN 
    Colours_3NF ON Colours_3NF.ColourID = Products_3NF.ColourID
JOIN 
    Locations_3NF ON Locations_3NF.LocationID =  Products_3NF.LocationID
JOIN 
    Transactions_3NF ON Transactions_3NF.Barcode = Products_3NF.Barcode
JOIN 
    Users_3NF ON Users_3NF.UserID = Transactions_3NF.UserID
JOIN 
    Carts_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
WHERE 
    Locations_3NF.StockCountry = 'South Africa' 
    AND SUBSTR(Carts_3NF.InvoiceDate, 6, 2) IN ('01', '03', '05', '07', '09','11')
    AND Colours_3NF.Colour = 'Black';

Query started at 11:37:11 PM GMT Daylight Time; Query executed in 0.00 m

,UserName,StockCountry,ColourID,Colour,Total,InvoiceDate
0,Cindy,South Africa,5,Black,1999.0,2020-03-31 0:00:00
1,Hester,South Africa,5,Black,189.0,2020-07-04 0:00:00
2,Conrad,South Africa,5,Black,93.0,2020-07-26 0:00:00
3,Carel,South Africa,5,Black,799.0,2020-05-04 0:00:00
4,Lacin,South Africa,5,Black,798.0,2020-05-18 0:00:00
5,Jeandre,South Africa,5,Black,179.0,2020-07-28 0:00:00
6,DIMPHO,South Africa,5,Black,1523.0,2020-07-02 0:00:00


In [136]:
%%read_sql

CREATE VIEW newest AS
SELECT 
    DISTINCT(Users_3NF.UserName), 
    Locations_3NF.StockCountry, 
    Products_3NF.ColourID, 
    Colours_3NF.Colour, 
    Carts_3NF.Total, 
    Carts_3NF.InvoiceDate
FROM 
    Products_3NF 
JOIN 
    Colours_3NF ON Colours_3NF.ColourID = Products_3NF.ColourID
JOIN 
    Locations_3NF ON Locations_3NF.LocationID =  Products_3NF.LocationID
JOIN 
    Transactions_3NF ON Transactions_3NF.Barcode = Products_3NF.Barcode
JOIN 
    Users_3NF ON Users_3NF.UserID = Transactions_3NF.UserID
JOIN 
    Carts_3NF ON Carts_3NF.CartID = Transactions_3NF.CartID
WHERE 
    Locations_3NF.StockCountry = 'South Africa' 
    AND SUBSTR(Carts_3NF.InvoiceDate, 6, 2) IN ('01', '03', '05', '07', '09','11')
    AND Colours_3NF.Colour = 'Black';

Query started at 12:23:39 AM GMT Daylight Time; Query executed in 0.00 m

In [137]:
%%read_sql

SELECT 
    SUM(Total) 
FROM 
    newest;

Query started at 12:24:15 AM GMT Daylight Time; Query executed in 0.00 m

,SUM(Total)
0,5580.0
